# Мария Студеникина СТФИ-122
## Вариант 2

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVR
from sklearn.impute import SimpleImputer

## Загрузка данных

In [2]:
# Загружаем данные из CSV файла
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
# Пример данных
df_train.head()

,TargetB,ID,TargetD,GiftCnt36,GiftCntAll,GiftCntCard36,GiftCntCardAll,GiftAvgLast,GiftAvg36,GiftAvgAll,...,PromCntCard36,PromCntCardAll,StatusCat96NK,StatusCatStarAll,DemAge,DemGender,DemHomeOwner,DemMedHomeValue,DemPctVeterans,DemMedIncome
0,0,14974,NaN,2,4,1,3,$17.00,$13.50,$9.25,...,8,13,A,0,NaN,F,U,$0,0,NaN
1,0,6294,NaN,1,8,0,3,$20.00,$20.00,$15.88,...,5,24,A,0,67.0,F,U,"$186,800",85,NaN
2,1,46110,$4.00,6,41,3,20,$6.00,$5.17,$3.73,...,11,22,S,1,NaN,M,U,"$87,600",36,"$38,750"
3,1,185937,$10.00,3,12,3,8,$10.00,$8.67,$8.50,...,6,16,E,1,NaN,M,U,"$139,200",27,"$38,942"
4,0,29637,NaN,1,1,1,1,$20.00,$20.00,$20.00,...,7,6,F,0,53.0,M,U,"$168,100",37,"$71,509"


In [4]:
# Пример данных
df_test.head()

,ID,GiftCnt36,GiftCntAll,GiftCntCard36,GiftCntCardAll,GiftAvgLast,GiftAvg36,GiftAvgAll,GiftAvgCard36,GiftTimeLast,...,PromCntCardAll,StatusCat96NK,StatusCatStarAll,DemAge,DemGender,DemHomeOwner,DemIncomeGroup,DemMedHomeValue,DemPctVeterans,DemMedIncome
0,3,2,12,1,6,$25.00,$22.50,$21.25,$20.00,18,...,25,A,1,69.0,F,H,5.0,"$101,400",33,NaN
1,6,7,22,4,6,$10.00,$5.71,$5.36,$5.25,5,...,22,A,0,NaN,M,U,4.0,"$57,600",24,NaN
2,9,5,16,3,8,$12.00,$12.80,$10.75,$13.00,16,...,35,S,1,NaN,M,U,5.0,"$44,200",0,NaN
3,11,1,15,0,4,$27.00,$27.00,$17.80,NaN,18,...,25,A,1,51.0,F,H,7.0,"$68,100",36,NaN
4,16,5,21,2,7,$25.00,$21.60,$20.10,$20.00,15,...,43,S,1,68.0,F,U,4.0,"$90,600",32,NaN


In [5]:
# Эта функция скопирована из задания 4
def plot_roc_curve(y_true, y_pred):
    fpr, tpr, threshold = metrics.roc_curve(y_true, y_pred)
    roc_auc = metrics.auc(fpr, tpr)
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.3f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [1, 0], 'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [6]:
# Предобработка данных (и тренировочных, и тестовых)

character_features_train = ['ID', 'StatusCat96NK', 'DemGender', 'DemHomeOwner', 'TargetB']
character_features_test = ['ID', 'StatusCat96NK', 'DemGender', 'DemHomeOwner']

# Удаляем номинальные переменные, так как с ними сложно работать
df_train.drop(columns = character_features_train, inplace = True)
df_test.drop(columns = character_features_test, inplace = True)

y_train = df_train
y_test = df_test
X_train = df_train.loc[:, sorted(list(set(df_train.columns)))]
X_test = df_test.loc[:, sorted(list(set(df_test.columns)))]

In [7]:
X_train

,DemAge,DemMedHomeValue,DemMedIncome,DemPctVeterans,GiftAvg36,GiftAvgAll,GiftAvgCard36,GiftAvgLast,GiftCnt36,GiftCntAll,...,GiftTimeFirst,GiftTimeLast,PromCnt12,PromCnt36,PromCntAll,PromCntCard12,PromCntCard36,PromCntCardAll,StatusCatStarAll,TargetD
0,NaN,$0,NaN,0,$13.50,$9.25,$17.00,$17.00,2,4,...,66,21,8,17,26,3,8,13,0,NaN
1,67.0,"$186,800",NaN,85,$20.00,$15.88,NaN,$20.00,1,8,...,92,26,14,35,79,5,5,24,0,NaN
2,NaN,"$87,600","$38,750",36,$5.17,$3.73,$5.00,$6.00,6,41,...,111,18,12,23,51,5,11,22,1,$4.00
3,NaN,"$139,200","$38,942",27,$8.67,$8.50,$8.67,$10.00,3,12,...,93,9,14,22,44,2,6,16,1,$10.00
4,53.0,"$168,100","$71,509",37,$20.00,$20.00,$20.00,$20.00,1,1,...,21,21,10,15,13,4,7,6,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9681,NaN,"$68,200","$45,909",38,$15.00,$15.00,$15.00,$10.00,2,2,...,24,17,19,29,27,8,12,11,0,$25.00
9682,NaN,"$71,900","$50,530",35,$13.33,$11.50,$10.00,$10.00,3,4,...,32,25,11,29,29,6,15,13,0,NaN
9683,71.0,"$115,700",NaN,32,$8.78,$7.71,$8.80,$10.00,9,14,...,55,15,13,35,46,6,17,18,1,$10.00
9684,NaN,"$106,300","$62,717",32,$12.50,$7.43,NaN,$5.00,2,7,...,56,6,15,34,42,6,6,16,0,NaN


In [8]:
X_test

,DemAge,DemIncomeGroup,DemMedHomeValue,DemMedIncome,DemPctVeterans,GiftAvg36,GiftAvgAll,GiftAvgCard36,GiftAvgLast,GiftCnt36,...,GiftCntCardAll,GiftTimeFirst,GiftTimeLast,PromCnt12,PromCnt36,PromCntAll,PromCntCard12,PromCntCard36,PromCntCardAll,StatusCatStarAll
0,69.0,5.0,"$101,400",NaN,33,$22.50,$21.25,$20.00,$25.00,2,...,6,92.0,18,23,41,60,8,18,25,1
1,NaN,4.0,"$57,600",NaN,24,$5.71,$5.36,$5.25,$10.00,7,...,6,92.0,5,20,38,54,9,18,22,0
2,NaN,5.0,"$44,200",NaN,0,$12.80,$10.75,$13.00,$12.00,5,...,8,91.0,16,27,48,92,11,21,35,1
3,51.0,7.0,"$68,100",NaN,36,$27.00,$17.80,NaN,$27.00,1,...,4,91.0,18,23,40,58,10,10,25,1
4,68.0,4.0,"$90,600",NaN,32,$21.60,$20.10,$20.00,$25.00,5,...,7,123.0,15,19,41,98,9,20,43,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96362,63.0,4.0,"$81,000",NaN,28,$20.00,$16.67,NaN,$20.00,1,...,1,75.0,19,10,24,33,4,4,12,0
96363,67.0,3.0,"$59,300",NaN,50,$8.00,$6.14,$8.00,$10.00,3,...,12,97.0,21,11,30,67,5,15,27,1
96364,24.0,5.0,"$67,400",NaN,32,$38.67,$38.67,$8.00,$100.00,3,...,2,29.0,19,16,29,28,6,12,10,1
96365,80.0,NaN,"$49,500",NaN,35,$10.00,$9.64,NaN,$10.00,2,...,7,120.0,18,17,36,76,6,6,31,1


In [10]:
# Заполняем пропущенные значения (NaN) самыми частыми значениями в колонке

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_train = pd.DataFrame(imp.fit_transform(X_train), columns = X_train.columns)

In [11]:
X_test = pd.DataFrame(imp.fit_transform(X_test), columns = X_test.columns)

ValueError: Shape of passed values is (96367, 21), indices imply (96367, 22)